# ML AB Testing

## Imports

In [1]:
import scipy
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
# from ABTesting import DfHelper

In [2]:
%matplotlib inline
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from df_helper import DfHelper

## Data reading

In [4]:
helper = DfHelper()
missing_values = ["n/a", "na", "undefined"]
df = helper.read_csv("../data/AdSmartABdata.csv")
df.head(5)

2021-07-24 07:05:07,949 — DfHelper — DEBUG — file read as csv


,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


## Linear

In [5]:
from config import Config

In [12]:
X_train = pd.read_csv(str(Config.FEATURES_PATH / "train_features.csv"))
y_train = pd.read_csv(str(Config.FEATURES_PATH / "train_labels.csv"))


In [13]:
X_train

,experiment,hour,date_of_week,date_of_week.1,month,device_make,browser
0,0,5,10,10,7,108,13
1,0,19,5,5,7,44,2
2,0,12,9,9,7,161,6
3,0,15,3,3,7,44,2
4,0,15,3,3,7,44,2
...,...,...,...,...,...,...,...
6456,0,15,3,3,7,44,2
6457,0,15,9,9,7,44,2
6458,1,6,6,6,7,173,3
6459,0,6,10,10,7,44,2


In [14]:
y_train

,aware
0,0
1,0
2,0
3,0
4,0
...,...
6456,0
6457,0
6458,0
6459,0


In [1]:
import xgboost


In [ ]:
xgboost??